# Huesos

## Importación de librerias

In [1]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os


## Definición de rutas

In [2]:
fracturados_train = r"C:\Users\David\PycharmProjects\first_project\Huesos\Bone_Fracture_Binary_Classification\Bone_Fracture_Binary_Classification\train\fractured"
fracturados_test = r"C:\Users\David\PycharmProjects\first_project\Huesos\Bone_Fracture_Binary_Classification\Bone_Fracture_Binary_Classification\test\fractured"
no_fracturados_train = r"C:\Users\David\PycharmProjects\first_project\Huesos\Bone_Fracture_Binary_Classification\Bone_Fracture_Binary_Classification\train\not fractured"
no_fracturados_test = r"C:\Users\David\PycharmProjects\first_project\Huesos\Bone_Fracture_Binary_Classification\Bone_Fracture_Binary_Classification\test\not fractured"

## Definición medidas

In [3]:
size = (200,200)

## Creación listas entrenamiento

In [74]:
x_train = []
x_test = []
y_train = []
y_test = []

Fracturado = 0 || No fracturado = 1

## Recorriendo directorio y añadiendo a las listas redimensionandolo

In [75]:
for i in os.listdir(fracturados_train):
    try:
        x_train.append(cv2.resize(cv2.cvtColor(cv2.imread(os.path.join(fracturados_train, i)), cv2.COLOR_BGR2GRAY), size)/255.0)
        y_train.append(0)
    except:
        pass

In [76]:
for i in os.listdir(fracturados_test):
    try:
        x_test.append(cv2.resize(cv2.cvtColor(cv2.imread(os.path.join(fracturados_test, i)), cv2.COLOR_BGR2GRAY), size)/255.0)
        y_test.append(0)
    except:
        print("error en fracturados test")

In [77]:
for i in os.listdir(no_fracturados_train):
    try:
        x_train.append(cv2.resize(cv2.cvtColor(cv2.imread(os.path.join(no_fracturados_train, i)), cv2.COLOR_BGR2GRAY), size)/255.0)
        y_train.append(1)
    except:
        pass

In [78]:
for i in os.listdir(no_fracturados_test):
    try:
        x_test.append(cv2.resize(cv2.cvtColor(cv2.imread(os.path.join(no_fracturados_test, i)), cv2.COLOR_BGR2GRAY), size)/255.0)
        y_test.append(1)
    except:
        print("error en no fracturados test")

In [79]:
print(len(x_train), len(y_train), len(x_test), len(y_test))

9243 9243 506 506


## Conversión a arrays de numpy

In [80]:
x_train = np.array(x_train, dtype="float16")

In [81]:
# Asegurar que las imágenes estén en escala de grises y tengan una dimensión de canal
x_train = x_train.reshape(-1, 200, 200, 1)

In [82]:
y_train = np.array(y_train, dtype = "float16")

In [83]:
x_test = np.array(x_test, dtype = "float16")

In [84]:
# Asegurar que las imágenes estén en escala de grises y tengan una dimensión de canal
x_test = x_test.reshape(-1,200, 200, 1)

In [85]:
y_test = np.array(y_test, dtype = "float16")

In [86]:
print(x_train.shape, y_train.shape)

(9243, 200, 200, 1) (9243,)


In [87]:
print(x_test.shape, y_test.shape)

(506, 200, 200, 1) (506,)


## Creación de modelo

In [88]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(200, 200, 1), dtype=tf.float16),
    tf.keras.layers.Conv2D(32,3, activation = "relu"),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(32,3, activation = "relu"),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(32,3, activation = "relu"),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(68, activation = "relu"),
    tf.keras.layers.Dense(128, activation = "relu"),
    tf.keras.layers.Dense(2, activation = "softmax")                  
])

In [89]:
model.compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy"]
)

## Barajar los datos

In [90]:
# Combinar x_train y y_train en un solo conjunto de datos
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

# Barajar los datos
dataset = dataset.shuffle(buffer_size=len(x_train))
dataset_preprocesado = dataset.batch(32).repeat(5)

## Entrenamiento de modelo

In [91]:
model.fit(dataset_preprocesado, epochs=5, steps_per_epoch=len(x_train) // 32)

Epoch 1/5
288/288 ━━━━━━━━━━━━━━━━━━━━ 134s 448ms/step - accuracy: 0.7535 - loss: 0.4488
Epoch 2/5
288/288 ━━━━━━━━━━━━━━━━━━━━ 127s 442ms/step - accuracy: 0.9820 - loss: 0.0644
Epoch 3/5
288/288 ━━━━━━━━━━━━━━━━━━━━ 125s 434ms/step - accuracy: 0.9940 - loss: 0.0202
Epoch 4/5
288/288 ━━━━━━━━━━━━━━━━━━━━ 124s 432ms/step - accuracy: 0.9947 - loss: 0.0134
Epoch 5/5
288/288 ━━━━━━━━━━━━━━━━━━━━ 122s 424ms/step - accuracy: 0.9957 - loss: 0.0179


In [92]:
model.evaluate(x_test, y_test)
prediction = model.predict(x_test)

16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 130ms/step - accuracy: 0.9940 - loss: 0.0343
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 140ms/step


## Pruebas de shape

In [44]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_train.dtype)
print(y_train.dtype)
print(x_test.dtype)
print(y_test.dtype)
print(dataset)

(5036, 200, 200, 1)
(5036,)
(276, 100, 100, 1)
(69,)
float16
float16
float16
float16
<_ShuffleDataset element_spec=(TensorSpec(shape=(200, 200, 1), dtype=tf.float16, name=None), TensorSpec(shape=(), dtype=tf.float16, name=None))>


## Probar datos

In [100]:
print(prediction[235:280])
print(y_test[235:280])

[[1.0000000e+00 4.4749850e-11]
 [1.0000000e+00 6.0282543e-11]
 [1.0000000e+00 2.4011185e-11]
 [2.7156761e-01 7.2843242e-01]
 [1.0441682e-02 9.8955834e-01]
 [3.9806516e-10 1.0000000e+00]
 [1.0441682e-02 9.8955834e-01]
 [5.2461593e-07 9.9999952e-01]
 [4.6939607e-04 9.9953067e-01]
 [6.0917944e-07 9.9999940e-01]
 [2.5495979e-08 1.0000000e+00]
 [1.9188386e-07 9.9999976e-01]
 [8.2519044e-05 9.9991751e-01]
 [6.3081440e-10 1.0000000e+00]
 [9.3080344e-06 9.9999070e-01]
 [2.9297832e-01 7.0702177e-01]
 [8.8046750e-07 9.9999917e-01]
 [1.1605373e-05 9.9998844e-01]
 [7.3824426e-06 9.9999261e-01]
 [2.6706894e-04 9.9973291e-01]
 [2.9297802e-01 7.0702201e-01]
 [2.9297832e-01 7.0702177e-01]
 [1.0441682e-02 9.8955834e-01]
 [5.4817833e-06 9.9999452e-01]
 [4.4571811e-06 9.9999559e-01]
 [1.0091454e-07 9.9999988e-01]
 [1.4801859e-02 9.8519820e-01]
 [6.7254547e-07 9.9999928e-01]
 [3.6738416e-05 9.9996328e-01]
 [4.0361101e-06 9.9999595e-01]
 [2.8725856e-04 9.9971277e-01]
 [6.3742642e-05 9.9993622e-01]
 [5.8941